In [ ]:
import pyodbc
import pandas as pd
import numpy as np

#  Conexión a **Azure SQL**
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

#  Conexión a **SQL Server LOCAL**
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'  
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

#  Consulta SQL en Azure SQL
SQL_QUERY = """
SELECT 
    s.CODE,                          -- Código de la venta
    s.Sales_Date,                    -- Fecha de la venta
    s.Customer_ID,                   -- ID del cliente
    s.Id_Producto,                    -- ID del producto
    s.PVP,                           -- Precio de venta al público
    s.IMPUESTOS,                     -- Impuestos
    s.COSTE_VENTA_NO_IMPUESTOS,      -- Coste de venta sin impuestos
    fp.FORMA_PAGO AS Forma_Pago,     -- Descripción de la forma de pago
    mv.MOTIVO_VENTA AS Motivo_Venta, -- Descripción del motivo de la venta
    t.TIENDA_ID,        -- ID de la tienda
    t.TIENDA_DESC AS Tienda,         -- Descripción de la tienda
    e.Car_Age,                       -- Edad del coche (Car Age)
    c.QUEJA,                         -- Queja (si existe)
    p.Modelo,                        -- Modelo del producto
    co.Costetransporte,              -- Coste de transporte
    co.GastosMarketing,              -- Gastos de marketing
    co.Margendistribuidor,           -- Margen del distribuidor
    co.Comisión_Marca,               -- Comisión de la marca (si está disponible)

    -- Cálculo del Margen Bruto
    ROUND(s.PVP * (co.Margen) * 0.01 * (1 - s.IMPUESTOS / 100), 2) AS Margen_eur_bruto,
    
    -- Cálculo del Margen Neto
    ROUND(
        s.PVP * (co.Margen) * 0.01 * (1 - s.IMPUESTOS / 100) 
        - s.COSTE_VENTA_NO_IMPUESTOS 
        - (co.Margendistribuidor * 0.01 + co.GastosMarketing * 0.01 - co.Comisión_Marca * 0.01) * s.PVP * (1 - s.IMPUESTOS / 100) 
        - co.Costetransporte, 
        2
    ) AS Margen_eur

FROM 
    DATAEX.[001_sales] s
LEFT JOIN 
    DATAEX.[010_forma_pago] fp ON s.FORMA_PAGO_ID = fp.FORMA_PAGO_ID  -- Join para forma de pago
LEFT JOIN 
    DATAEX.[009_motivo_venta] mv ON s.MOTIVO_VENTA_ID = mv.MOTIVO_VENTA_ID  -- Join para motivo de venta
LEFT JOIN 
    DATAEX.[011_tienda] t ON s.TIENDA_ID = t.TIENDA_ID  -- Join para tienda
LEFT JOIN 
    DATAEX.[018_edad] e ON s.CODE = e.CODE  -- Join para Car_Age
LEFT JOIN 
    DATAEX.[008_cac] c ON s.CODE = c.CODE  -- Join para quejas
LEFT JOIN 
    DATAEX.[006_producto] p ON s.Id_Producto = p.Id_Producto  -- Join para producto
LEFT JOIN 
    DATAEX.[007_costes] co ON p.Modelo = co.Modelo;  -- Join para costes


"""

# 🔹 Nombre de la tabla en SQL Server Local
NEW_TABLE_NAME = "DATAEX.FACT_SALES"

try:
    #  Conectar a Azure SQL
    print(f"Conectando a Azure SQL...")
    conn_azure = pyodbc.connect(azure_conn_str)
    
    # 🔹 Ejecutar la consulta en Azure SQL
    print(f"Ejecutando consulta en Azure SQL...")
    df = pd.read_sql(SQL_QUERY, conn_azure)

    if df.empty:
        print(f" La consulta no devolvió resultados. No se creará la tabla en SQL Server Local.")
    else:
        print(f"   - Datos extraídos: {df.shape[0]} filas")



        #  Convertir NaN en columnas numéricas a 0
        df = df.fillna(0)

        #  Convertir valores numéricos problemáticos
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)  # Reducir precisión
        
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)  # Evitar valores fuera de rango
        
        #  Conectar a SQL Server Local
        print(f"Conectando a SQL Server Local...")
        conn_local = pyodbc.connect(local_conn_str)
        
        with conn_local.cursor() as cursor:
            # 🔹 Eliminar la tabla si ya existe
            drop_table_sql = f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}"
            cursor.execute(drop_table_sql)
            conn_local.commit()
            print(f"   - Tabla eliminada si existía.")

            # 🔹 Crear la tabla en SQL Server Local con tipos de datos ajustados
            create_table_sql = f"""
            CREATE TABLE {NEW_TABLE_NAME} (
                {', '.join([
                    f'[{col}] FLOAT' if df[col].dtype == np.float32 
                    else f'[{col}] INT' if df[col].dtype == np.int32 
                    else f'[{col}] NVARCHAR(255)' for col in df.columns
                ])}
            );
            """
            cursor.execute(create_table_sql)
            conn_local.commit()
            print(f" Tabla {NEW_TABLE_NAME} creada correctamente en SQL Server Local.")

            # Insertar los datos en SQL Server Local
            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"

            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()

            print(f" {df.shape[0]} filas insertadas en {NEW_TABLE_NAME}.")

except Exception as e:
    print(f" Error: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("\n ¡Proceso completado!")


Conectando a Azure SQL...
Ejecutando consulta en Azure SQL...


C:\Users\josit\AppData\Local\Temp\ipykernel_12432\2694264098.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL_QUERY, conn_azure)


   - Datos extraídos: 3652 filas
Conectando a SQL Server Local...
   - Tabla eliminada si existía.
 Tabla DATAEX.DIM_TIEMPO creada correctamente en SQL Server Local.
 3652 filas insertadas en DATAEX.DIM_TIEMPO.

 ¡Proceso completado!
